# Fraud Detection

This notebook shows how to use Amazon Sagemaker Processsing to prepare the data. 


## Sagemaker Initial Setup
The below code is used to get the S3 Bucket name configured for SageMaker

In [1]:
! pip install -qU sagemaker
import sagemaker
from sagemaker import get_execution_role

role = get_execution_role()
sess = sagemaker.Session()
bucket = sess.default_bucket() 
print("S3 bucket name: ", bucket)

/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
S3 bucket name:  sagemaker-ap-south-1-123450495660



The below code is used to get the pre-built Docker Image URL for scikit-learn container stored in Amazon Elastic Container Registry (Amazon ECR)

In [2]:
import boto3

region = boto3.session.Session().region_name
accountRegionMap =	{
  "us-west-1": "746614075791","us-west-2": "246618743249","us-east-1": "683313688378",
  "us-east-2": "257758044811","ap-northeast-1": "354813040037","ap-northeast-2": "354813040037",
  "ap-southeast-1": "121021644041","ap-southeast-2": "783357654285","ap-south-1": "720646828776",
  "eu-west-1": "141502667606","eu-west-2": "764974769150","eu-central-1": "492215442770","ca-central-1": "341280168497"
}

account_id = accountRegionMap[region]
ecr_repository = "sagemaker-scikit-learn"
tag = ":0.23-1-cpu-py3"

uri_suffix = "amazonaws.com"
if region in ["cn-north-1", "cn-northwest-1"]:
    uri_suffix = "amazonaws.com.cn"
ecr_repository_uri = "{}.dkr.ecr.{}.{}/{}".format(
    account_id, region, uri_suffix, ecr_repository + tag
)

print("SageMaker processing repository uri: ", ecr_repository_uri)

SageMaker processing repository uri:  720646828776.dkr.ecr.ap-south-1.amazonaws.com/sagemaker-scikit-learn:0.23-1-cpu-py3


## Amazon Sagemaker Data Preprocessing 

In [3]:
# The "source" field is used for the dataset and the "destination" is used to store the prepared data
source = 's3://'+bucket+'/AutoInsuranceFraudDetection/DataSet/insurance_claims.csv'
destination = 's3://'+bucket+'/AutoInsuranceFraudDetection/Results/DataProcessing'

In [4]:
%%writefile AutoInsuranceFraudProcessing.py
#This block of code generates a file "AutoInsuranceFraudProcessing.py" which has the code to process the data

import argparse
import os
import warnings

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelBinarizer, KBinsDiscretizer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.compose import make_column_transformer

from sklearn.exceptions import DataConversionWarning
from sklearn.preprocessing import LabelEncoder

warnings.filterwarnings(action="ignore", category=DataConversionWarning)

if __name__ == "__main__":
    #get arguments
    parser = argparse.ArgumentParser()
    args, _ = parser.parse_known_args()
    print("Received arguments {}".format(args))
    
    #get the input data
    input_data_path = os.path.join("/opt/ml/processing/input", "insurance_claims.csv")
    print("Reading input data from {}".format(input_data_path))
    df = pd.read_csv(input_data_path)
    df = pd.DataFrame(data=df)
    print(df.head())

    #replacing ? with nan for the columns
    df['police_report_available']=df['police_report_available'].replace('?',np.nan)
    #dropping the unnecessary rows
    df=df.dropna(subset=['police_report_available'])
    
    #drop unnecessary columns
    df=df.drop(['months_as_customer'],axis=1)
    
    #now deal with the categorical features
    #for the columns insured_sex and fraud_reported
    le=LabelEncoder()
    for i in list(df.columns):
        if df[i].dtype=='object':
            df[i]=le.fit_transform(df[i])
    
    #final preprocessed data
    print(df.head())
    train_features_output_path = os.path.join("/opt/ml/processing/output", "preprocessed_data.csv")
    df.to_csv(train_features_output_path, index=False)
    print("done")
    
    

Overwriting AutoInsuranceFraudProcessing.py


### Run Preprocessing job with Amazon SageMaker Processing

The script we have defined at `AutoInsuranceFraudProcessing.py` performs data preprocessing transformations on the raw data. The preproceesing involes replacing values, droping rows, dropping columns and categorical encoding

In [5]:
from sagemaker.processing import ScriptProcessor, ProcessingInput, ProcessingOutput

script_processor = ScriptProcessor(command=['python3'],
                                   image_uri=ecr_repository_uri,
                                   role=role,
                                   instance_count=1,
                                   instance_type='ml.m4.2xlarge')

script_processor.run(code='AutoInsuranceFraudProcessing.py',
                     inputs=[ProcessingInput(source=source,
                                             destination='/opt/ml/processing/input')],
                     outputs=[ProcessingOutput(output_name="preprocessed_data.csv", destination=destination,
                                               source='/opt/ml/processing/output')])




Job Name:  sagemaker-scikit-learn-2021-09-14-15-27-34-775
Inputs:  [{'InputName': 'input-1', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-ap-south-1-123450495660/AutoInsuranceFraudDetection/DataSet/insurance_claims.csv', 'LocalPath': '/opt/ml/processing/input', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'code', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://sagemaker-ap-south-1-123450495660/sagemaker-scikit-learn-2021-09-14-15-27-34-775/input/code/AutoInsuranceFraudProcessing.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'preprocessed_data.csv', 'AppManaged': False, 'S3Output': {'S3Uri': 's3://sagemaker-ap-south-1-123450495660/AutoInsuranceFraudDetection/Results/DataProcessing', 'LocalPath': '/opt/ml/processing/output', 'S3

### View Results of Data Preprocessing

Once the preprocessing job is complete, we can take a look at the contents of the S3 bucket to see the transformed data.

In [19]:
preprocessing_job_description = script_processor.jobs[-1].describe()

output_config = preprocessing_job_description["ProcessingOutputConfig"]
for output in output_config["Outputs"]:
    print(output)
    preprocessed_data = output["S3Output"]["S3Uri"]
    print(preprocessed_data)


{'OutputName': 'preprocessed_data.csv', 'S3Output': {'S3Uri': 's3://sagemaker-ap-south-1-123450495660/AutoInsuranceFraudDetection/Results/DataProcessing', 'LocalPath': '/opt/ml/processing/output', 'S3UploadMode': 'EndOfJob'}, 'AppManaged': False}
s3://sagemaker-ap-south-1-123450495660/AutoInsuranceFraudDetection/Results/DataProcessing


Once the training is completed, the training instances are automatically saved and SageMaker stores the trained model and evaluation results to a location in S3.